In [1]:
jcvi_variables = dict(
  # age on phase 1 reference date
    vax_jcvi_age_1=patients.age_as_of(
        ref_age_1,
        return_expectations={
            "int": {"distribution": "population_ages"},
            "rate": "universal",
        },
    ),

    # age on phase 2 reference date
    vax_jcvi_age_2=patients.age_as_of(
        ref_age_2,
        return_expectations={
            "int": {"distribution": "population_ages"},
            "rate": "universal",
        },
    ),

    vax_cat_jcvi_group=patients.categorised_as(
        dict_jcvi,
        return_expectations={
            "rate": "universal",
            "incidence": 1,
            "category": { 
                "ratios": ratio_jcvi 
                }
        },
    ),

    ### NEED TO DISCUSS CONDITIONS FOR PREGNANCY (FEMALE AND <50)
    ### WAS ADDED INITIALLY TO AVOID CODING ERRORS, BUT NOT VERY INCLUSIVE SO CONSIDER REVISING
    #### Pregnancy or Delivery codes recorded (for deriving JCVI group)
    # # date of last pregnancy code in 36 weeks before ref_cev
    preg_group=patients.satisfying(
        """
        (preg_36wks_date AND cov_cat_sex = 'F' AND vax_jcvi_age_1 < 50) AND
        (pregdel_pre_date <= preg_36wks_date OR NOT pregdel_pre_date)
        """,
        preg_36wks_date=patients.with_these_clinical_events(
            preg_primis,
            returning="date",
            find_last_match_in_period=True,
            between=[days(ref_cev, -252), days(ref_cev, -1)],
            date_format="YYYY-MM-DD",
        ),
        # date of last delivery code recorded in 36 weeks before elig_date
        pregdel_pre_date=patients.with_these_clinical_events(
            pregdel_primis,
            returning="date",
            find_last_match_in_period=True,
            between=[days(ref_cev, -252), days(ref_cev, -1)],
            date_format="YYYY-MM-DD",
        ),
    ),

    #### clinically extremely vulnerable group variables
    cev_group=patients.satisfying(
        "severely_clinically_vulnerable AND NOT less_vulnerable",

        # SHIELDED GROUP - first flag all patients with "high risk" codes
        severely_clinically_vulnerable=patients.with_these_clinical_events(
            shield_primis,
            returning="binary_flag",
            on_or_before=days(ref_cev, -1),
            find_last_match_in_period=True,
        ),

        # find date at which the high risk code was added
        severely_clinically_vulnerable_date=patients.date_of(
            "severely_clinically_vulnerable",
            date_format="YYYY-MM-DD",
        ),

        # NOT SHIELDED GROUP (medium and low risk) - only flag if later than 'shielded'
        less_vulnerable=patients.with_these_clinical_events(
            nonshield_primis,
            between=["severely_clinically_vulnerable_date + 1 day", days(ref_cev, -1)],
        ),
        return_expectations={"incidence": 0.01},
    ),

    #### at-risk group variables
    # Asthma Diagnosis code
    astdx=patients.with_these_clinical_events(
        ast_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.05},
    ),
            
    # asthma
    asthma_group=patients.satisfying(
        """
        astadm OR
        (astdx AND astrxm1 AND astrxm2 AND astrxm3)
        """,
        # day before date at which at risk group became eligible
        # Asthma Admission codes
        astadm=patients.with_these_clinical_events(
            astadm_primis,
            returning="binary_flag",
            on_or_before=days(ref_ar, -1),
        ),
        # Asthma systemic steroid prescription code in month 1
        astrxm1=patients.with_these_medications(
            astrx_primis,
            returning="binary_flag",
            between=[days(ref_ar, -31), days(ref_ar, -1)],
        ),
        # Asthma systemic steroid prescription code in month 2
        astrxm2=patients.with_these_medications(
            astrx_primis,
            returning="binary_flag",
            between=[days(ref_ar, -61), days(ref_ar, -32)],
        ),
        # Asthma systemic steroid prescription code in month 3
        astrxm3=patients.with_these_medications(
            astrx_primis,
            returning="binary_flag",
            between=[days(ref_ar, -91), days(ref_ar, -62)],
        ),
        return_expectations={"incidence": 0.01},
    ),

    # Chronic Respiratory Disease other than asthma
    resp_group=patients.with_these_clinical_events(
        resp_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.02},
    ),

    # Chronic Neurological Disease including Significant Learning Disorder
    cns_group=patients.with_these_clinical_events(
        cns_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.01},
    ),

    # diabetes
    diab_group=patients.satisfying(
        """
        (NOT dmres_date AND diab_date) OR
        (dmres_date < diab_date)
        """,
        diab_date=patients.with_these_clinical_events(
            diab_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        dmres_date=patients.with_these_clinical_events(
            dmres_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        return_expectations={"incidence": 0.01},
    ),

    # severe mental illness codes
    sevment_group=patients.satisfying(
        """
        (NOT smhres_date AND sev_mental_date) OR
        smhres_date < sev_mental_date
        """,
        # Severe Mental Illness codes
        sev_mental_date=patients.with_these_clinical_events(
            sev_mental_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        # Remission codes relating to Severe Mental Illness
        smhres_date=patients.with_these_clinical_events(
            smhres_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        return_expectations={"incidence": 0.01},
    ),

    # Chronic heart disease codes
    chd_group=patients.with_these_clinical_events(
        chd_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.01},
    ),

    # Chronic kidney disease diagnostic codes
    ckd_group=patients.satisfying(
        """
            ckd OR
            (ckd15_date AND 
            (ckd35_date >= ckd15_date) OR (ckd35_date AND NOT ckd15_date))
        """,
        # Chronic kidney disease codes - all stages
        ckd15_date=patients.with_these_clinical_events(
            ckd15_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        # Chronic kidney disease codes-stages 3 - 5
        ckd35_date=patients.with_these_clinical_events(
            ckd35_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        # Chronic kidney disease diagnostic codes
        ckd=patients.with_these_clinical_events(
            ckd_primis,
            returning="binary_flag",
            on_or_before=days(ref_ar, -1),
        ),
        return_expectations={"incidence": 0.01},
    ),

    # Chronic Liver disease codes
    cld_group=patients.with_these_clinical_events(
        cld_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.01},
    ),

    # immunosuppressed
    immuno_group=patients.satisfying(
        "immrx OR immdx", 
        # immunosuppression diagnosis codes
        immdx=patients.with_these_clinical_events(
            immdx_primis,
            returning="binary_flag",
            on_or_before=days(ref_ar, -1),
        ),
        # Immunosuppression medication codes
        immrx=patients.with_these_medications(
            immrx_primis,
            returning="binary_flag",
            between=[days(ref_ar, -6*30), days(ref_ar, -1)],
        ),
        return_expectations={"incidence": 0.01},
    ),

    # Asplenia or Dysfunction of the Spleen codes
    spln_group=patients.with_these_clinical_events(
        spln_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.01},
    ),

    # Wider Learning Disability
    learndis_group=patients.with_these_clinical_events(
        learndis_primis,
        returning="binary_flag",
        on_or_before=days(ref_ar, -1),
        return_expectations={"incidence": 0.01},
    ),

    # severe obesity
    sevobese_group=patients.satisfying(
        """
        (sev_obesity_date AND NOT bmi_date) OR
        (sev_obesity_date > bmi_date) OR
        bmi_value_temp >= 40
        """,
        bmi_stage_date=patients.with_these_clinical_events(
            bmi_stage_primis,
            returning="date",
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        sev_obesity_date=patients.with_these_clinical_events(
            sev_obesity_primis,
            returning="date",
            find_last_match_in_period=True,
            ignore_missing_values=True,
            between= ["bmi_stage_date", days(ref_ar, -1)],
            date_format="YYYY-MM-DD",
        ),
        bmi_date=patients.with_these_clinical_events(
            bmi_primis,
            returning="date",
            ignore_missing_values=True,
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            date_format="YYYY-MM-DD",
        ),
        bmi_value_temp=patients.with_these_clinical_events(
            bmi_primis,
            returning="numeric_value",
            ignore_missing_values=True,
            find_last_match_in_period=True,
            on_or_before=days(ref_ar, -1),
            return_expectations={
                "float": {"distribution": "normal", "mean": 25, "stddev": 5},
            },
        ),
        return_expectations={"incidence": 0.01},
    ),

    # at risk group
    atrisk_group=patients.satisfying(
             """
             immuno_group OR
             ckd_group OR
             resp_group OR
             diab_group OR
             cld_group OR
             cns_group OR
             chd_group OR
             spln_group OR
             learndis_group OR
             sevment_group OR
             sevobese_group 
            """,
            return_expectations = {
            "incidence": 0.01,
            },
    ),

    # Patients in long-stay nursing and residential care
    longres_group=patients.with_these_clinical_events(
        longres_primis,
        returning="binary_flag",
        on_or_before=days(start_date, -1),
        return_expectations={"incidence": 0.01},
    ),
    

    # vaccine eligibility dates
    vax_date_eligible=patients.categorised_as(
       dict_elig,
        return_expectations={
            "category": {"ratios": 
            ratio_elig
            },
            "incidence": 1,
        },
    ),

)


NameError: name 'patients' is not defined

In [2]:
from cohortextractor import codelist_from_csv, combine_codelists, codelist

#Covid
covid_codes = codelist_from_csv(
    "codelists/user-RochelleKnight-confirmed-hospitalised-covid-19.csv",
    system="icd10",
    column="code",
)

covid_primary_care_positive_test = codelist_from_csv(
    "codelists/opensafely-covid-identification-in-primary-care-probable-covid-positive-test.csv",
    system="ctv3",
    column="CTV3ID",
)

covid_primary_care_code = codelist_from_csv(
    "codelists/opensafely-covid-identification-in-primary-care-probable-covid-clinical-code.csv",
    system="ctv3",
    column="CTV3ID",
)

covid_primary_care_sequalae = codelist_from_csv(
    "codelists/opensafely-covid-identification-in-primary-care-probable-covid-sequelae.csv",
    system="ctv3",
    column="CTV3ID",
)
#Ethnicity
opensafely_ethnicity_codes_6 = codelist_from_csv(
    "codelists/opensafely-ethnicity.csv",
    system="ctv3",
    column="Code",
    category_column="Grouping_6",
)

primis_covid19_vacc_update_ethnicity = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-eth2001.csv",
    system="snomed",
    column="code",
    category_column="grouping_6_id",
)
#Smoking
smoking_clear = codelist_from_csv(
    "codelists/opensafely-smoking-clear.csv",
    system="ctv3",
    column="CTV3Code",
    category_column="Category",
)

smoking_unclear = codelist_from_csv(
    "codelists/opensafely-smoking-unclear.csv",
    system="ctv3",
    column="CTV3Code",
    category_column="Category",
)



# BMI
bmi_obesity_snomed_clinical = codelist_from_csv(
    "codelists/user-elsie_horne-bmi_obesity_snomed.csv",
    system="snomed",
    column="code",
)

bmi_obesity_icd10 = codelist_from_csv(
    "codelists/user-elsie_horne-bmi_obesity_icd10.csv",
    system="icd10",
    column="code",
)

bmi_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-bmi.csv",
    system="snomed",
    column="code",
)


# Total Cholesterol
cholesterol_snomed = codelist_from_csv(
    "codelists/opensafely-cholesterol-tests-numerical-value.csv",
    system="snomed",
    column="code",
)

# HDL Cholesterol
hdl_cholesterol_snomed = codelist_from_csv(
    "codelists/bristol-hdl-cholesterol.csv",
    system="snomed",
    column="code",
)
# Carer codes
carer_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-carer.csv",
    system="snomed",
    column="code",
)

# No longer a carer codes
notcarer_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-notcarer.csv",
    system="snomed",
    column="code",
)
# Wider Learning Disability
learndis_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-learndis.csv",
    system="snomed",
    column="code",
)
# Employed by Care Home codes
carehome_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-carehome.csv",
    system="snomed",
    column="code",
)

# Employed by nursing home codes
nursehome_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-nursehome.csv",
    system="snomed",
    column="code",
)

# Employed by domiciliary care provider codes
domcare_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-domcare.csv",
    system="snomed",
    column="code",
)

# Patients in long-stay nursing and residential care
longres_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-longres.csv",
    system="snomed",
    column="code",
)
# High Risk from COVID-19 code
shield_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-shield.csv",
    system="snomed",
    column="code",
)

# Lower Risk from COVID-19 codes
nonshield_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-nonshield.csv",
    system="snomed",
    column="code",
)

#For JCVI groups
# Pregnancy codes 
preg_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-preg.csv",
    system="snomed",
    column="code",
)

# Pregnancy or Delivery codes
pregdel_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-pregdel.csv",
    system="snomed",
    column="code",
)
# All BMI coded terms
bmi_stage_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-bmi_stage.csv",
    system="snomed",
    column="code",
)
# Severe Obesity code recorded
sev_obesity_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-sev_obesity.csv",
    system="snomed",
    column="code",
)
# Asthma Diagnosis code
ast_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-ast.csv",
    system="snomed",
    column="code",
)

# Asthma Admission codes
astadm_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-astadm.csv",
    system="snomed",
    column="code",
)

# Asthma systemic steroid prescription codes
astrx_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-astrx.csv",
    system="snomed",
    column="code",
)
# Chronic Respiratory Disease
resp_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-resp_cov.csv",
    system="snomed",
    column="code",
)
# Chronic Neurological Disease including Significant Learning Disorder
cns_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-cns_cov.csv",
    system="snomed",
    column="code",
)

# Asplenia or Dysfunction of the Spleen codes
spln_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-spln_cov.csv",
    system="snomed",
    column="code",
)
# Diabetes diagnosis codes
diab_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-diab.csv",
    system="snomed",
    column="code",
)
# Diabetes resolved codes
dmres_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-dmres.csv",
    system="snomed",
    column="code",
)
# Severe Mental Illness codes
sev_mental_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-sev_mental.csv",
    system="snomed",
    column="code",
)

# Remission codes relating to Severe Mental Illness
smhres_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-smhres.csv",
    system="snomed",
    column="code",
)

# Chronic heart disease codes
chd_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-chd_cov.csv",
    system="snomed",
    column="code",
)

# Chronic kidney disease diagnostic codes
ckd_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-ckd_cov.csv",
    system="snomed",
    column="code",
)

# Chronic kidney disease codes - all stages
ckd15_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-ckd15.csv",
    system="snomed",
    column="code",
)

# Chronic kidney disease codes-stages 3 - 5
ckd35_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-ckd35.csv",
    system="snomed",
    column="code",
)

# Chronic Liver disease codes
cld_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-cld.csv",
    system="snomed",
    column="code",
)
# Immunosuppression diagnosis codes
immdx_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-immdx_cov.csv",
    system="snomed",
    column="code",
)

# Immunosuppression medication codes
immrx_primis = codelist_from_csv(
    "codelists/primis-covid19-vacc-uptake-immrx.csv",
    system="snomed",
    column="code",
)

##Quality assurance codes 

prostate_cancer_snomed_clinical = codelist_from_csv(
    "codelists/user-RochelleKnight-prostate_cancer_snomed.csv",
    system="snomed",
    column="code",
)
prostate_cancer_icd10 = codelist_from_csv(
    "codelists/user-RochelleKnight-prostate_cancer_icd10.csv",
    system="icd10",
    column="code",
)
pregnancy_snomed_clinical = codelist_from_csv(
    "codelists/user-RochelleKnight-pregnancy_and_birth_snomed.csv",
    system="snomed",
    column="code",
)


FileNotFoundError: [Errno 2] No such file or directory: 'codelists/user-RochelleKnight-confirmed-hospitalised-covid-19.csv'